In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
x_frame1_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame1_0.5_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame2_0.5_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame3_0.5_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame_noise_0.5_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

659


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
x_adj=np.array(x_adj)

In [11]:
def calculate_g(data):
    temp2=[]
    for vs_out in data:
        a = np.sum(vs_out, axis=0).reshape(-1, 1)
        b = np.sum(vs_out != 0, axis=1).reshape(-1, 1)
        temp2.append(np.concatenate((a, b), axis=1))
        
    return temp2

In [12]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
659


In [13]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(calculate_g, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 1.5521104335784912 seconds ---


In [14]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [15]:
x_g=np.array(x_g)


In [16]:
print(x_g.shape)

(31615, 56, 2)


In [17]:
x_g[0]

array([[3.07617288e-02, 1.00000000e+00],
       [1.22697324e-01, 1.50000000e+01],
       [7.40531007e-02, 1.10000000e+01],
       [6.17527697e-02, 7.00000000e+00],
       [4.85433343e-02, 4.00000000e+00],
       [5.14107707e-02, 1.00000000e+01],
       [4.69595670e-02, 1.00000000e+01],
       [4.87198250e-02, 8.00000000e+00],
       [3.05176181e-02, 4.00000000e+00],
       [3.79232171e-02, 4.00000000e+00],
       [5.50537909e-02, 8.00000000e+00],
       [5.20833633e-02, 3.00000000e+00],
       [5.20577410e-02, 9.00000000e+00],
       [4.66308794e-02, 2.00000000e+00],
       [2.38919725e-01, 2.30000000e+01],
       [6.43636468e-02, 4.00000000e+00],
       [5.49316806e-02, 4.00000000e+00],
       [8.39030248e-02, 3.00000000e+00],
       [1.52792126e-01, 1.40000000e+01],
       [7.17773638e-02, 2.00000000e+00],
       [6.98453839e-02, 6.00000000e+00],
       [9.96298644e-02, 1.20000000e+01],
       [8.43463877e-02, 7.00000000e+00],
       [7.83691806e-02, 4.00000000e+00],
       [5.102543

In [18]:
x_g_flat=np.array([np.array(i.T.flatten()).ravel() for i in x_g])


In [19]:
x_g_flat[0]

array([3.07617288e-02, 1.22697324e-01, 7.40531007e-02, 6.17527697e-02,
       4.85433343e-02, 5.14107707e-02, 4.69595670e-02, 4.87198250e-02,
       3.05176181e-02, 3.79232171e-02, 5.50537909e-02, 5.20833633e-02,
       5.20577410e-02, 4.66308794e-02, 2.38919725e-01, 6.43636468e-02,
       5.49316806e-02, 8.39030248e-02, 1.52792126e-01, 7.17773638e-02,
       6.98453839e-02, 9.96298644e-02, 8.43463877e-02, 7.83691806e-02,
       5.10254306e-02, 1.26713210e-01, 8.48002616e-02, 7.64509529e-02,
       1.10422570e-01, 7.48291316e-02, 1.19873077e-01, 2.55858005e-01,
       1.01834137e-01, 1.54570736e-01, 2.48589931e-01, 3.51126903e-01,
       2.46370087e-01, 4.21966673e-01, 7.01200119e-01, 5.01761100e-01,
       4.79809796e-01, 4.22261616e-01, 4.12907988e-01, 3.65789865e-01,
       4.94404203e-01, 1.66388678e-01, 2.54245899e-01, 2.17771514e-01,
       6.62272635e-02, 4.41894831e-02, 5.56640925e-02, 7.05729967e-02,
       3.66811617e-02, 4.79053774e-02, 1.53808794e-02, 3.10590921e-02,
      

In [20]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g_flat, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [21]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(22130, 112)
(4742, 112)
(4743, 112)


In [22]:
import tensorflow as tf


2023-08-13 10:07:49.226910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 10:07:49.397962: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 10:07:49.402073: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [23]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 10:08:11.769886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 10:08:11.769919: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 10:08:11.769939: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0155): /proc/driver/nvidia/version does not exist
2023-08-13 10:08:11.770655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [24]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 112, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 56, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 56, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 28, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 100)               8

In [25]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F8.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [27]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
692/692 [==============================] - 4s 4ms/step - loss: 0.1108 - acc: 0.9620 - val_loss: 0.0248 - val_acc: 0.9943
Epoch 2/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0226 - acc: 0.9946 - val_loss: 0.0187 - val_acc: 0.9949
Epoch 3/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0203 - acc: 0.9945 - val_loss: 0.0182 - val_acc: 0.9954
Epoch 4/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0158 - acc: 0.9963 - val_loss: 0.0245 - val_acc: 0.9941
Epoch 5/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0144 - acc: 0.9966 - val_loss: 0.0157 - val_acc: 0.9964
Epoch 6/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0139 - acc: 0.9963 - val_loss: 0.0186 - val_acc: 0.9960
Epoch 7/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0139 - acc: 0.9968 - val_loss: 0.0139 - val_acc: 0.9975
Epoch 8/30
692/692 [==============================] - 3s 4ms/step - loss: 0.

Epoch 29/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0039 - acc: 0.9988 - val_loss: 0.0104 - val_acc: 0.9979
Epoch 30/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0056 - acc: 0.9984 - val_loss: 0.0241 - val_acc: 0.9949


In [28]:
model3.save_weights('/home/nvekariy/signal_code/New_neural_code_SNR_0.5_h/F8.h5')


In [29]:
model3.load_weights('/home/nvekariy/signal_code/New_neural_code_SNR_0.5_h/F8.h5')


In [30]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

149/149 [==============================] - 0s 1ms/step


In [31]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1004    0    0    2]
 [   2  943   15    2]
 [   0    1  955    1]
 [   1    0    0 1817]]


In [32]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [33]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

99.49399114484504
99.41381527154401
99.49399114484504
99.4981218474723
